In [1]:
#调用函数
from selenium import webdriver
import pandas as pd
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException

In [2]:
#翻页函数
def next_page(driver):
    try:
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
    except NoSuchElementException:
        print("NoSuchElementException")

In [3]:
#爬取函数
def scrape_page_data(driver):
    data = []
    #解析爬取内容
    houses = driver.find_elements(By.CSS_SELECTOR, 'dl.clearfix')

    for house in houses:
        try:
            title = house.find_element(By.CSS_SELECTOR, 'h4.clearfix a').get_attribute('title')
            price_text = house.find_element(By.CSS_SELECTOR, 'dd.price_right').text
            price = price_text.split()[0]
            price_per_sqm = price_text.split()[1]
            house_info = house.find_element(By.CSS_SELECTOR, 'p.tel_shop').text
            info_parts = [x.strip() for x in house_info.split('|')]
            area_text = house.find_element(By.CSS_SELECTOR, 'p.add_shop').text
            name = area_text.split()[0]
            area = area_text.split()[1]
            try:
                metro = house.find_element(By.CSS_SELECTOR, 'span.bg_none.icon_dt').text
            except:
                metro = '无'

            item = {
                '标题': title,
                '小区名': name,
                '总价': price,
                '单价': price_per_sqm,
                '户型': info_parts[0],
                '面积': info_parts[1],
                '层型': info_parts[2],
                '朝向': info_parts[3],
                '建成时间': info_parts[4],
                '区域': area,
                '交通信息': metro
            }

            data.append(item)
        except Exception as e:
            print(f'解析条目时出错：{str(e)}')
            continue
    return data

In [4]:
#保存数据
def save_data(data, filename, is_first_page=False):
    df = pd.DataFrame(data)
    df.to_csv(filename,
              mode='a' if not is_first_page else 'w',
              header=is_first_page,
              index=False,
              encoding='utf_8_sig')

In [5]:
#定义爬取要求
def scrape_real_estate_data(url, filename, max_pages=20):
    driver = webdriver.Edge()
    driver.get(url)
    sleep(20)

    first_page = True
    for i in range(max_pages):
        page_data = scrape_page_data(driver)
        save_data(page_data, filename, is_first_page=first_page)
        print(f"第{i+1}页数据处理完成")
        first_page = False
        if i < max_pages - 1:
            next_page(driver)
            sleep(3)
    
    print(f"前{max_pages}页处理完成")
    print(f"爬取网址：{url}")
    print(f"总页数：{max_pages}")
    driver.quit()

In [ ]:
#爬取数据
scrape_real_estate_data('https://tj.zu.fang.com/house-a041-b0967/', 'price1-tianjin-balitai.csv', max_pages=20)

第1页数据处理完成
第2页数据处理完成


KeyboardInterrupt: 